# <font color='white'>**Libraries**</font>

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import tensorflow as tf
import sklearn
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import regularizers
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing import image
from tensorflow import keras
import imageio
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
AUTOTUNE = tf.data.AUTOTUNE
from tqdm import tqdm
from numpy import asarray
import random
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from tensorflow import keras

# <font color='red'>**Helper functions**</font>

# <font color='red'>**Networks**</font>

In [ ]:
def transfer_learning(arquitectura, base_model, IMG_HEIGHT, IMG_WIDTH, num_clases):
    """
    Creates a new transfer learning model by adding custom layers on top of the given base model.

    Parameters:
    arquitectura (str): The architecture of the base model to be used. Can be 'ResNet18', 'Vgg16', or any other value.
    base_model: The pre-trained base model to which custom layers will be added.
    IMG_HEIGHT (int): The target height for the input images.
    IMG_WIDTH (int): The target width for the input images.
    num_clases (int): The number of output classes for the classification task.

    Returns:
    model (tensorflow.keras.models.Model): A new transfer learning model with custom layers on top of the base model.

    Notes:
    - The function allows transfer learning using different pre-trained base models like 'ResNet18', 'Vgg16', etc.
    - The base model's lower layers are frozen to prevent further training and fine-tuning.
    - The custom layers are added on top of the base model to adapt it for the specific classification task.
    - The activation function for the last layer is set to 'softmax' for multi-class classification tasks.
    """
    print("making transfer learning...")

    # Freeze the base_model lower layers
    for layer in base_model.layers[:10]:
        layer.trainable = False
    for layer in base_model.layers[10:]:
        layer.trainable = True

    # Create a new model on top based on the specified architecture
    if arquitectura == 'Vgg16':
        inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
        x = base_model(inputs)
        x = GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Flatten(name='flatten')(x)
        x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01), name='fc1')(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01), name='fc2')(x)
        predictions = tf.keras.layers.Dense(num_clases, activation='softmax', name='predictions')(x)
        model = tf.keras.models.Model(inputs=inputs, outputs=predictions)
    else:
        inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
        x = base_model(inputs)
        x = GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Flatten(name='flatten')(x)
        x = tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01), name='fc1')(x)
        x = tf.keras.layers.Dropout(0.5)(x)
        x = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01), name='fc2')(x)
        predictions = Dense(num_clases, activation='softmax')(x)
        model = Model(inputs=inputs, outputs=predictions)

    return model


In [ ]:
def make_model(arquitectura, HEIGHT, WIDTH):
    """
    Creates a base model for transfer learning based on the specified architecture.

    Parameters:
    arquitectura (str): The architecture of the base model to be created. Can be 'MobileNet', 'Vgg16', or 'EfficientNetV2B0'.
    HEIGHT (int): The target height for the input images.
    WIDTH (int): The target width for the input images.

    Returns:
    base_model (tensorflow.keras.models.Model): A pre-trained base model suitable for transfer learning.

    Notes:
    - The function uses Keras applications to load pre-trained base models for transfer learning.
    - The 'MobileNet', 'Vgg16', and 'EfficientNetV2B0' models are available for selection.
    - The input shape for the base model is set to (HEIGHT, WIDTH, 3) to match the target image size and 3 color channels.
    - The 'weights' parameter is set to 'imagenet' to use pre-trained weights on the ImageNet dataset.
    - The 'include_top' parameter is set to False to exclude the top classification layers of the base model.
    """
    print("loading : ", arquitectura)

    # Create a base model for transfer learning based on the specified architecture
    if arquitectura == 'MobileNet':
        base_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False,
                                                     input_shape=(HEIGHT, WIDTH, 3))
    elif arquitectura == 'Vgg16':
        base_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False,
                                                 input_shape=(HEIGHT, WIDTH, 3))
    elif arquitectura == 'EfficientNetV2B0':
        base_model = tf.keras.applications.EfficientNetV2B0(weights='imagenet', include_top=False,
                                                            input_shape=(HEIGHT, WIDTH, 3))

    return base_model


## Data generator

In [ ]:
def make_generator(df_train, df_val, HEIGHT, WIDTH, tipo, batch_size):
    """
    Creates image generators for training and validation data from the given DataFrames.

    Parameters:
    df_train (pandas.DataFrame): DataFrame containing the training data with two columns, 'path' and 'label'.
    df_val (pandas.DataFrame): DataFrame containing the validation data with two columns, 'path' and 'label'.
    HEIGHT (int): The target height for the input images.
    WIDTH (int): The target width for the input images.
    tipo (str): The class mode for the generator. Can be 'binary' for binary classification or 'categorical' for multi-class.
    batch_size (int): The batch size for the data generator.

    Returns:
    train_generator (tensorflow.python.keras.preprocessing.image.DataFrameIterator): A data generator for training data.
    valid_generator (tensorflow.python.keras.preprocessing.image.DataFrameIterator): A data generator for validation data.

    Notes:
    - The function creates two image data generators, one for training data and one for validation data.
    - The training data generator reads data from the DataFrame 'df_train', and the validation data generator reads
      data from the DataFrame 'df_val'.
    - The 'tipo' parameter determines the class mode for the generator ('binary' for binary classification or
      'categorical' for multi-class classification).
    - The 'target_size' parameter is set to (HEIGHT, WIDTH) to resize the images to the specified dimensions.
    - The 'seed' parameter is set to 42 for reproducibility of random transformations applied to the images.
    - The 'shuffle' parameter is set to True to shuffle the data during each epoch.
    """
    # Generator for training data
    datagen = ImageDataGenerator(rescale=1. / 255)#, rotation_range= 20, brightness_range = [0.2, 0.8], zoom_range=0.2, horizontal_flip=True,
                                 #vertical_flip=True)

    train_generator = datagen.flow_from_dataframe(directory=None, 
                                                  dataframe=df_train,
                                                  x_col='path', 
                                                  y_col='label', 
                                                  target_size=(HEIGHT, WIDTH),
                                                  class_mode=tipo, 
                                                  batch_size=batch_size,
                                                  seed=42,
                                                  shuffle=True)

    # Generator for validation data
    val_datagen = ImageDataGenerator(rescale=1. / 255)

    valid_generator = val_datagen.flow_from_dataframe(directory=None,
                                                      dataframe=df_val,
                                                      x_col='path',
                                                      y_col='label',
                                                      batch_size=batch_size,
                                                      seed=42,
                                                      shuffle=True,
                                                      class_mode=tipo,
                                                      target_size=(HEIGHT, WIDTH))

    return train_generator, valid_generator


## Train

In [ ]:
#this cell is for wide training using different nets
def train(arquitectura, group, HEIGHT, WIDTH, df_train, df_val, lr, train_epochs, batch_size, loss, 
          tipo, clases):
    
    base_model = make_model(arquitectura, HEIGHT, WIDTH)
    # making trasfer learning
    finetune_model = transfer_learning(arquitectura, base_model, HEIGHT, WIDTH, clases)
    # creating generadores
    print("getting generators...")
    train_generator, valid_generator = make_generator(df_train, df_val, HEIGHT,WIDTH, tipo, batch_size)

    # training steps over each dataset
    NUM_EPOCHS = train_epochs
    num_train_images = len(df_train)#*augmenting_factor
    STEP_SIZE_TRAIN=num_train_images//train_generator.batch_size
    STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
    
    print("amount of epocas: ", NUM_EPOCHS)
    print("amount of images to train: ", num_train_images)
    print("amount of images to valid: ", valid_generator.n)

    
    save_path = "../models/clasification/" + arquitectura + group + ".h5"
        
    callback_list = [tf.keras.callbacks.EarlyStopping(patience=4, monitor='val_acc', mode='max'),                                     
                    tf.keras.callbacks.ModelCheckpoint(filepath=save_path,
                                                      monitor = 'val_acc',
                                                      verbose=1,
                                                      save_best_only=True,
                                                      mode = 'max',
                                                      save_weights_only=False,
                                                      save_freq='epoch')]
    #------new from here
    # Define the L2 regularization loss
    l2_loss = tf.add_n(finetune_model.losses)
    #------ until here
    
    solver = Adam(learning_rate=lr)
    print("class weights")
    total = df_train.shape[0]
    weights = (total/df_train.groupby('label').count().values)/3
    class_weight = {0:weights[0][0], 1:weights[1][0], 2:weights[2][0]}
    print(class_weight)
    
    print("compiling model")
    finetune_model.compile(solver, loss=loss, metrics=[tf.keras.metrics.CategoricalAccuracy(name='acc')])
    
    print("model training...")
    history = finetune_model.fit(train_generator,
                                 steps_per_epoch = STEP_SIZE_TRAIN,
                                 epochs=NUM_EPOCHS,
                                 validation_data=valid_generator,
                                 validation_steps=STEP_SIZE_VALID,
                                 class_weight=class_weight,
                                 callbacks=callback_list)
    
    return finetune_model, history, train_generator, valid_generator 

In [ ]:
#group = "dry"
#train_df, val_df, test_df = load_dataframes(group)
#train_df.groupby('label').count()

In [ ]:
#loading the wet data
group = "Seca"

if group == "Humeda":
    df = pd.read_csv("../data/rgb/classification/augmented/Humeda.csv")
else:
    print("leyendo info de secas")
    df = pd.read_csv("../data/rgb/classification/augmented/Seca.csv")
    
df.columns = ['path', 'label']

#making the train and test splits
train_df, test_df = train_test_split(df, test_size=0.2, random_state=14)

print(train_df.groupby('label').count())
print(test_df.groupby('label').count())


In [ ]:
#print("lengh of train: {}, lengh of valid: {}, lengh of test: {}".format(len(train_df), len(val_df), len(test_df)))

In [ ]:
#current_train_df = pd.concat([train_df, val_df], axis=0)
#len(current_train_df)
#current_train_df.groupby('label').count()

In [ ]:
#this cell is for a general classification knowledge
nets = ['Vgg16', 'MobileNet', 'EfficientNetV2B0']

num_clases=3

HEIGHT, WIDTH = 224, 224
activacion_final = 'softmax'
x_col_name = 'path'
y_col_name = 'label'
tipo = 'categorical'
batch_size = 16
train_epochs = 30
lr=0.00001
loss='categorical_crossentropy' 

for net in nets:
    arquitectura = net 
    np.random.seed(42)
    tf.random.set_seed(42)
    tf.keras.backend.clear_session()

    finetune_model, history, train_generator, valid_generator = train(arquitectura, group, HEIGHT, WIDTH, 
                                                                      train_df, test_df, lr, train_epochs,
                                                                      batch_size, loss, tipo, num_clases)

## Testing
### Making generator

In [ ]:
def make_generator(df_test, HEIGHT, WIDTH, tipo, batch_size):
    
    test_datagen=ImageDataGenerator()
    
    test_generator=test_datagen.flow_from_dataframe(directory=None,
                                                    dataframe=df_test,
                                                    x_col='path',
                                                    y_col='label',
                                                    batch_size=batch_size,
                                                    seed=42,
                                                    shuffle=False,
                                                    class_mode=tipo,
                                                    target_size=(HEIGHT,WIDTH))

    return test_generator

In [ ]:
HEIGHT, WIDTH = 224, 224
tipo = 'categorical'
batch_size = 16

In [ ]:
test_gen = make_generator(test_df, HEIGHT, WIDTH, tipo, batch_size)

In [ ]:
#loading the 
name = "EfficientNetV2B0"+ "Seca"
trained_model_path = '../models/clasification/' + name + '.h5'
model = keras.models.load_model(trained_model_path, compile=True)

In [ ]:
#Confution Matrix and Classification Report
test_gen.reset()
logits = model.predict(test_gen, test_df.shape[0] // batch_size+1)
y_pred_class = np.argmax(logits, axis=1)
#predicted_class_probab=np.max(logits,axis=1)

target_names = ['CuNi1', 'CuNi2', 'CuNi3']   
  
print('Confusion Matrix')
print(confusion_matrix(test_gen.classes, y_pred_class))
print('Classification Report')
print(classification_report(test_gen.classes, y_pred_class, target_names=target_names))

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

target_names = ['CuNi1', 'CuNi2', 'CuNi3']   
cm = confusion_matrix(test_gen.classes, y_pred_class, normalize='true')
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_names)
disp = disp.plot(include_values=True, cmap=plt.cm.Blues, xticks_rotation='horizontal', values_format='.2f')

In [ ]:
AUC = tf.keras.metrics.AUC()
AUC.update_state(test_gen.classes, y_pred_class)
AUC.result()